In [1]:

import sys
import math
import time
from random import randint,choice, shuffle, random
from statistics import mode
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
# from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import *
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn import tree

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix 

def specificity(y_true, y_predicted):
 tp,fp,fn,tn = confusion_matrix(y_true, y_predicted).ravel()
 s=tn/(tn+fp)
 return s

my_scorer = make_scorer(specificity, greater_is_better=True)





In [2]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/M_dataset/creditcard.csv')
data.drop('Time',axis=1,inplace=True)


In [3]:
data

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-4.280584,1.421100,-3.908229,2.942946,-0.076205,-2.002526,-2.874155,-0.856005,0.963674,-3.235439,3.341802,-7.562860,-1.847443,-11.571423,-1.481556,-6.180919,-9.285186,-3.921722,0.685511,-0.198963,-0.140062,-0.907720,-0.680108,-0.349170,0.056276,-1.149923,-1.809886,0.723051,1.10,1
1,2.257125,-1.487231,-0.788434,-1.626205,-1.388520,-0.484561,-1.309927,-0.045722,-1.230185,1.749482,0.283023,-0.537333,-0.261271,-0.186231,-0.731001,-0.182653,0.144075,0.629653,0.225567,-0.456688,-0.133463,0.076276,0.208021,-0.373993,-0.264262,-0.175708,0.012096,-0.060707,18.60,0
2,-0.583880,0.749139,2.489315,0.727643,-0.501264,0.231825,0.077561,0.222471,0.272318,-0.602338,-0.358267,0.580004,0.310034,-0.538042,0.151595,-0.446991,0.137113,-0.506306,-0.526871,-0.118078,0.084302,0.429332,-0.083170,0.403401,-0.162060,-0.454096,0.013724,0.073031,9.99,0
3,-0.380097,0.947642,-0.772336,-0.443717,0.873774,-0.821833,1.007168,-0.211991,0.550825,0.265309,0.751080,-0.574200,-1.617997,-0.706972,0.311639,0.244523,0.057290,1.358522,0.131358,0.234876,0.143695,0.927982,-0.217375,-0.577027,-0.277710,-0.139845,0.437612,0.112958,36.96,0
4,0.657478,-1.285148,0.943871,0.401469,-1.579852,-0.115491,-0.471521,0.171981,1.040693,-0.375914,0.935344,0.855607,-1.017754,-0.105701,-0.886167,0.092820,0.028775,-0.046239,0.602073,0.390129,-0.023686,-0.472047,-0.122505,0.597774,-0.046644,0.888341,-0.091673,0.048901,265.16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.905274,0.080784,1.547836,2.570243,-0.100630,2.017006,-0.840341,0.709949,-0.016073,0.282370,0.391973,0.949713,0.475365,-0.386157,0.701073,-0.766096,0.874924,-2.319315,-2.521562,-0.262425,0.064652,0.504713,0.201644,-0.618860,-0.043392,0.062233,0.117348,0.026562,6.08,0
9996,1.181350,-0.559179,0.369252,0.462396,-0.411321,0.667445,-0.531420,0.047032,-0.743940,0.632558,-1.365323,0.501385,1.796559,-0.488333,0.396258,-1.498662,-0.215539,0.956308,-1.072387,-0.305485,-0.358005,-0.482800,-0.157808,-0.961184,0.528067,-0.254456,0.080471,0.036632,84.31,0
9997,-0.060135,0.832088,0.664149,-0.561490,1.044491,-0.540045,1.271010,-0.452163,-0.328619,-0.409502,-1.337485,0.022807,0.708579,-0.148464,-0.076929,-0.247611,-0.566867,-0.521969,0.819025,0.066957,-0.289636,-0.562028,-0.187347,-0.628397,-0.390286,0.249422,-0.070092,-0.077364,0.89,0
9998,-19.438377,-17.164140,-8.610240,4.073813,0.664572,-0.502303,6.491054,-3.981043,4.666933,5.316694,1.718154,-0.394894,0.601822,-2.370572,1.559224,1.910168,-2.391402,-1.105333,2.011332,-12.070717,-4.508257,1.132565,0.013644,0.459717,1.182505,0.161029,-9.543518,15.870474,0.76,0


In [4]:
col_names=list(data.columns)
Att_names=col_names[:-1]
Class_names=data["Class"].unique()

In [5]:
n_att = len(Att_names)
n_att


 

29

In [6]:

train_data, test_data = train_test_split(data, test_size=0.3)
train_x=train_data.iloc[:,:-1].values
train_y=train_data["Class"].values
test_x=test_data.iloc[:,:-1].values
test_y=test_data["Class"].values


LogisticRegression


In [7]:

# Logestic LogisticRegression

clf=GridSearchCV(LogisticRegression(max_iter=100,n_jobs=-1,class_weight='balanced'),scoring=my_scorer,param_grid={'C':[0.1]})






In [8]:
#Constants
#pheromone concentration influence 
alpha = 0.5 
#MI influence
beta = 0.5
#pheromone evoparation
rho = 0.1
#Pheromone diposite level
q = 0.5
epsilon = 0.001

In [9]:
clf.fit(train_x,train_y)
clf.score(test_x,test_y)


0.6049382716049383

In [10]:

# number of ants and number of iterations
tset = list(range(n_att))
n_ants = 10
n_iter = 10
ants=[[] for x in range(n_ants)]
ac_ants=[0 for x in range(n_ants)]
best_tour = tset.copy()
best_ac=clf.fit(train_x[:,best_tour],train_y).score(test_x[:,best_tour],test_y)
#Intializing Pheromone
ph_trail = []
for i in range(n_att):
    p_r = [0.5 for j in range(n_att)]
    p_r[i] = 0
    ph_trail.append(p_r)
t_time_s = time.time()
t_t_time = 0
for i in range(n_iter):
    print("Iteration No :", i)
    at_time_s = time.time()
    for a in range(n_ants):
        ants[a]=[choice(best_tour)]
        while True:
            change=False
            mic = clf.fit(train_x[:,ants[a]],train_y).score(test_x[:,ants[a]],test_y)
            next_pos = {}
            max_next = []
            for at in range(n_att):
                if at not in ants[a]:
                    ph=0
                    hi = clf.fit(train_x[:,ants[a]+[at]],train_y).score(test_x[:,ants[a]+[at]],test_y)- mic
                    hi= 0 if hi<=0 else hi
                    for x_pos in ants[a]:
                        ph+=ph_trail[x_pos][at]
                    ph=ph/len(ants[a])
                    t_value = (hi ** beta) * (ph ** alpha)
                    if t_value>0:
                        max_next.append((t_value,at))
            if(len(max_next)):    
                pr=randint(0,len(max_next)-1)
                n_at = max_next[randint(0, pr)][1]
                ants[a].append(n_at)
                change=True
            if change==False:
                break
    for a in range(n_ants):
        ascore=clf.fit(train_x[:,ants[a]],train_y).score(test_x[:,ants[a]],test_y)
        ac_ants[a]=(ascore)
        bscore=best_ac
        if ascore>bscore:
            best_tour = ants[a].copy()
            best_ac=ac_ants[a]
        elif ascore==bscore:
            if len(ants[a])<len(best_tour):
                best_tour = ants[a].copy()
                best_ac=ac_ants[a]
    for a in range(n_ants):
        for k in ants[a]:
            for j in ants[a]:
                if k != j:
                    ph_trail[k][j] += ph_trail[k][j] *  ac_ants[a]/ len(ants[a])
                    if ph_trail[k][j]>1:
                        ph_trail[k][j]=1
    for k in best_tour:
        for j in best_tour:
            if k != j:
                ph_trail[k][j] += ph_trail[k][j] *  best_ac/ len(ants[a])
                if ph_trail[k][j]>1:
                    ph_trail[k][j]=1
    for j in range(n_att):
        for k in range(n_att):
            if k!=j:
                ph_trail[j][k]*=(1-rho)
    at_time_e = time.time()
    tempset=set()
    print("###########Iteration : ",i,"############")
    for a in range(n_ants):
        print("Sol of ant ",a," is :",ants[a],"     Specificity:",ac_ants[a])
    print("Sol of best ant till now is :",[Att_names[x] for x in best_tour],"     Specificity:",best_ac)
    print("Best_Sol length :  ", len(best_tour))
    print("Time Taken by iteration ", i, " is :", at_time_e - at_time_s)
t_time_e = time.time()
print("Total time by ACO (in sec): ", t_time_e - t_time_s)
print("Best_Sol  :  ", [Att_names[x] for x in best_tour])
print("Best_Sol length :  ", len(best_tour))
print("Specificity of Best_ant solution is :",best_ac)


Iteration No : 0
###########Iteration :  0 ############
Sol of ant  0  is : [11, 1, 2, 5, 0, 9, 6, 3, 15, 18, 7, 13, 19, 26, 17]      Specificity: 0.6606334841628959
Sol of ant  1  is : [28, 5, 6, 16, 0, 2, 8, 7, 1, 9, 10, 17, 12, 15, 24]      Specificity: 0.6731707317073171
Sol of ant  2  is : [11, 15, 1, 4, 12, 5, 0, 2, 6, 3, 9, 10, 13, 19]      Specificity: 0.6681818181818182
Sol of ant  3  is : [15, 6, 0, 13, 2, 8, 5, 14, 9, 23]      Specificity: 0.7014925373134329
Sol of ant  4  is : [24, 3, 4, 8, 11, 13, 10, 12, 17, 7, 5, 0, 18, 15, 20, 9, 22, 1]      Specificity: 0.6681818181818182
Sol of ant  5  is : [8, 0, 3, 7, 1, 4, 2, 16, 6, 21, 19, 11, 9, 14, 17, 26, 27]      Specificity: 0.6141078838174274
Sol of ant  6  is : [20, 26, 15, 1, 5, 10, 2, 8, 6, 13, 12, 19, 18]      Specificity: 0.6926829268292682
Sol of ant  7  is : [13, 9, 2, 6, 8, 15, 1, 26, 12]      Specificity: 0.7305699481865285
Sol of ant  8  is : [28, 17, 0, 3, 16, 1, 6, 12, 2, 9, 11, 13, 5, 20, 22]      Specificity: 0